# Day 10: Cathode-Ray Tube

## Part 1

ou avoid the ropes, plunge into the river, and swim to shore.

The Elves yell something about meeting back up with them upriver, but the river is too loud to tell exactly what they're saying. They finish crossing the bridge and disappear from view.

Situations like this must be why the Elves prioritized getting the communication system on your handheld device working. You pull it out of your pack, but the amount of water slowly draining from a big crack in its screen tells you it probably won't be of much immediate use.

Unless, that is, you can design a replacement for the device's video system! It seems to be some kind of cathode-ray tube screen and simple CPU that are both driven by a precise clock circuit. The clock circuit ticks at a constant rate; each tick is called a cycle.

Start by figuring out the signal being sent by the CPU. The CPU has a single register, X, which starts with the value 1. It supports only two instructions:

addx V takes two cycles to complete. After two cycles, the X register is increased by the value V. (V can be negative.)
noop takes one cycle to complete. It has no other effect.
The CPU uses these instructions in a program (your puzzle input) to, somehow, tell the screen what to draw.

Consider the following small program:
```
noop
addx 3
addx -5
```
Execution of this program proceeds as follows:

At the start of the first cycle, the noop instruction begins execution. During the first cycle, X is 1. After the first cycle, the noop instruction finishes execution, doing nothing.
At the start of the second cycle, the addx 3 instruction begins execution. During the second cycle, X is still 1.
During the third cycle, X is still 1. After the third cycle, the addx 3 instruction finishes execution, setting X to 4.
At the start of the fourth cycle, the addx -5 instruction begins execution. During the fourth cycle, X is still 4.
During the fifth cycle, X is still 4. After the fifth cycle, the addx -5 instruction finishes execution, setting X to -1.
Maybe you can learn something by looking at the value of the X register throughout execution. For now, consider the signal strength (the cycle number multiplied by the value of the X register) during the 20th cycle and every 40 cycles after that (that is, during the 20th, 60th, 100th, 140th, 180th, and 220th cycles).

For example, consider this larger program: (Omitted to save space but viewable here: https://adventofcode.com/2022/day/10.)

The interesting signal strengths can be determined as follows:

During the 20th cycle, register X has the value 21, so the signal strength is 20 * 21 = 420. (The 20th cycle occurs in the middle of the second addx -1, so the value of register X is the starting value, 1, plus all of the other addx values up to that point: 1 + 15 - 11 + 6 - 3 + 5 - 1 - 8 + 13 + 4 = 21.)
During the 60th cycle, register X has the value 19, so the signal strength is 60 * 19 = 1140.
During the 100th cycle, register X has the value 18, so the signal strength is 100 * 18 = 1800.
During the 140th cycle, register X has the value 21, so the signal strength is 140 * 21 = 2940.
During the 180th cycle, register X has the value 16, so the signal strength is 180 * 16 = 2880.
During the 220th cycle, register X has the value 18, so the signal strength is 220 * 18 = 3960.
The sum of these signal strengths is 13140.

Find the signal strength during the 20th, 60th, 100th, 140th, 180th, and 220th cycles. What is the sum of these six signal strengths?

In [2]:
import inspect

def parse_input(filename):
    instructions = []
    with open(filename) as file:
        lines = file.read().splitlines()
    for line in lines:
        line_split = line.split(" ")
        if line_split[0] == "noop":
            instructions.append("noop")
        elif line_split[0] == "addx":
            instructions.append(["addx", int(line_split[1])])
        else:
            message = inspect.cleandoc(f"""
                                Invalid instruction: {line_split}""")
            raise Exception(message)
    return instructions


def execute_instruction(current_X_value, instruction):
    X_list_to_return = []
    if instruction == "noop":
        X_list_to_return.append(current_X_value)
    elif instruction[0] == "addx":
        X_list_to_return.append(current_X_value)
        X_list_to_return.append(current_X_value + instruction[1])
    return X_list_to_return


def create_X_list(instructions):
    X_history = [1]
    for instruction in instructions:
        new_X_list = execute_instruction(X_history[-1], instruction)
        for X in new_X_list:
            X_history.append(X)
    return X_history

def main(filename):
    instructions = parse_input(filename)
    X_list = create_X_list(instructions)
    running_sum = 0
    for index_position in range(19, len(X_list) + 1, 40):
        signal_strength = X_list[index_position] * (index_position + 1)
        running_sum += signal_strength
    return running_sum
        

#%%
if __name__ == "__main__":
    result = main("input.txt")
    print(f"Sum of signal strengths: {result}.")

Sum of signal strengths: 11720.


In [3]:
def get_sprite_position(middle):
    message = inspect.cleandoc(f"""
                               middle must be of type {type(0)}, not of type
                               {type(middle)}""")
    assert isinstance(middle, int), message
    return [middle - 1, middle, middle + 1]

def get_image(X_list):
    to_print = ""
    for pixel_column in range(6): # CRT is 6 high.
        for pixel_row in range(40): # CTR is 40 wide.
            pixel_index = 40 * pixel_column + pixel_row
            sprite_position = get_sprite_position(X_list[pixel_index])
            if pixel_row in sprite_position:
                to_print += "#"
            else:
                    to_print += "."
        if pixel_column != 6:
            to_print += "\n"
    return to_print

if __name__ == "__main__":
    X_list = create_X_list(parse_input("input.txt"))
    print(get_image(X_list))

####.###...##..###..####.###...##....##.
#....#..#.#..#.#..#.#....#..#.#..#....#.
###..#..#.#....#..#.###..#..#.#.......#.
#....###..#....###..#....###..#.......#.
#....#.#..#..#.#.#..#....#....#..#.#..#.
####.#..#..##..#..#.####.#.....##...##..



> I read ERCREPCJ.